## *PhotoDissociation Region Toolbox Notebooks*
---------------------------------------------------------------

# Example 2: Introduction to ModelSets

PDRT supports a variety of PDR models to be used to fit your data.  The current default are the Wolfire/Kaufman 2006 constant density models for metallicities z=1. There are also a WK2006 z=3 and z=0.1 sets and a new 2020 z=1 model set, all with limited spectral lines. Models are stored as ratios of intensities as a function of radiation field $G_0$ and hydrogen nucleus volume density $n$. We are expanding the 2020 model set now to include a  wider range of lines and metallicities.  Any PDR models
can be used if they are stored in the [correct FITS format](https://docs.google.com/document/d/1hPSUip5RK6nPmqrL8iw3qkX69DVLBv0sAuqac18pLT8/edit?usp=sharing).  We are currently working with Marcus Rollig to import the KosmaTau models.
 

In [1]:
from pdrtpy.modelset import ModelSet
from pdrtpy import pdrutils as utils

/home/mpound/anaconda3/lib/python3.7/site-packages/astropy/table/column.py:1161: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = getattr(super(), op)(other)


In [2]:
# check that your notebooks align with your pdrtpy version
utils.check_nb()

Your PDRT notebooks version matches your pdrtpy version -- hooray!


### list() describes all the ModelSets available. 
(Note `kosmatau` is a placeholder for now to demostrate that external models can be imported.)

In [3]:
ModelSet.list()

  name   version  z                                       description                                      
-------- ------- --- --------------------------------------------------------------------------------------
wk2006   2006    1.0 Wolfire/Kaufman 2006 constant density models                                          
wk2006   2006    3.0 Wolfire/Kaufman 2006 constant density models                                          
wk2020   2020    1.0 Wolfire/Kaufman 2020 constant density models, small set, Av=7, no freeze-out          
smc      2006    0.1 Wolfire/Kaufman 2006 limited set of constant density models for Small Magellanic Cloud
kosmatau X       1.0 Test set KOSMA Tau models, Z=1                                                        


### Instantiate the 2020 Wolfire/Kaufman models for z=1. 

In [4]:
m = ModelSet("wk2006",z=1)

### Show the model spectral lines and ratios that are available in this ModelSet
`supported_lines` and `supported_ratios` is an astropy Table, invoke *show_in_notebook()* to see all the rows.  There may be more ratios than lines because not all components of a ratio may be available as separate models.

In [5]:
m.supported_lines.show_in_notebook()

idx,title,intensity label
,null,
0,[C I] 609 $\mu$m,CI_609
1,[C II] 158 $\mu$m,CII_158
2,[O I] 145 $\mu$m,OI_145
3,[O I] 63 $\mu$m,OI_63
4,CO(J=1-0),CO_10
5,CO(J=2-1),CO_21
6,CO(J=4-3),CO_43
7,CO(J=5-4),CO_54
8,CO(J=6-5),CO_65


In [6]:
m.supported_ratios.show_in_notebook()

idx,title,denominator,ratio label
0,[O I] 145 $\mu$m / [O I] 63 $\mu$m,OI_63,OI_145/OI_63
1,[O I] 145 $\mu$m / [C II] 158 $\mu$m,CII_158,OI_145/CII_158
2,[O I] 63 $\mu$m / [C II] 158 $\mu$m,CII_158,OI_63/CII_158
3,[C II] 158 $\mu$m / [C I] 609 $\mu$m,CI_609,CII_158/CI_609
4,[C I] 370 $\mu$m / [C I] 609 $\mu$m,CI_609,CI_370/CI_609
5,[C II] 158 $\mu$m / CO(J=1-0),CO_10,CII_158/CO_10
6,[C II] 158 $\mu$m / CO(J=3-2),CO_32,CII_158/CO_32
7,[C II] 158 $\mu$m / CO(J=6-5),CO_65,CII_158/CO_65
8,[C II] 158 $\mu$m / I$_{FIR}$,FIR,CII_158/FIR
9,[C I] 609 $\mu$m / CO(J=1-0),CO_10,CI_609/CO_10


Get the Wolfire/Kaufman 2006 models.  Every model has a one-line description. You can query a ModelSet to find which spectral lines (your observations) are supported and which model ratios it has for your lines.

In [7]:
t = ModelSet("wk2006",z=1)
print(t.description)

mylines =["CII_158","OI_145","FIR","CI_609","CO_65"]
print("There are %d ratios in the %s ModelSet for the lines %s:"%(t.ratiocount(mylines),t.description,mylines))
print(t.model_ratios(mylines))

# If you want to see the actual FITS file names. The root of the path is where your pdrtpy is installed.
for p in t.find_files(mylines):
    print(p)

Wolfire/Kaufman 2006 constant density models, Z=1.0
There are 7 ratios in the Wolfire/Kaufman 2006 constant density models, Z=1.0 ModelSet for the lines ['CII_158', 'OI_145', 'FIR', 'CI_609', 'CO_65']:
['CII_158/FIR', 'CII_158/CI_609', 'CII_158/CO_65', 'OI_145/CII_158', 'OI_145+CII_158/FIR', 'CI_609/FIR', 'CI_609/CO_65']
('CII_158/FIR', 'wolfirekaufman/version2006/constant_density/z=1/CII158_FIR.fits')
('CII_158/CI_609', 'wolfirekaufman/version2006/constant_density/z=1/ciici609web.fits')
('CII_158/CO_65', 'wolfirekaufman/version2006/constant_density/z=1/CII158_CO65.fits')
('OI_145/CII_158', 'wolfirekaufman/version2006/constant_density/z=1/o145ciiweb.fits')
('OI_145+CII_158/FIR', 'wolfirekaufman/version2006/constant_density/z=1/firoi145web.fits')
('CI_609/FIR', 'wolfirekaufman/version2006/constant_density/z=1/ci609firweb.fits')
('CI_609/CO_65', 'wolfirekaufman/version2006/constant_density/z=1/cico65web.fits')


### How to access a single model
This example returns the model for CII 158 micron/CO(4-3) intensity ratio. The access key is the tag in the `ratio label` in the supported ratios table above.  The model is returned as a `pdrtpy.measurement.Measurement`

In [8]:
cico = t.get_model("CI_609/CO_43")
print(cico.id,cico.data.min(),cico.data.max())

CI_609/CO_43 0.0019551122 3577.9814


### How to access multiple models
Suppose you want to find all the models that use some combination of CO(J=1-0), CII 158 micron, and OI 145 micron. This example gets both intensity and ratio models, though you can specify `model_type='intensity'` or `model_type='ratio'` to get one or the other.  The models are returns as a dictionary with the keys set to the model IDs.

In [9]:
mods = t.get_models(["CII_158","OI_145", "CO_10"],model_type='both')

#### The returned models are three intensity models and two ratio models

In [10]:
mods.keys()

dict_keys(['OI_145', 'CO_10', 'CII_158', 'CII_158/CO_10', 'OI_145/CII_158'])

In [11]:
# average value of one of the models
print(mods['CII_158/CO_10'].data.mean())

11072.426
